In [1]:
# !pip install bert-score
# !pip install rouge
# !pip install nltk

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')  # Might be needed for other operations


[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import argparse
import json
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge import Rouge
import numpy as np
import statistics
import pandas as pd
import bert_score

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import single_meteor_score

import torch
from transformers import BertTokenizer, BertForMaskedLM


In [4]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [6]:
def read_result_csv(file_path):
    result_df = pd.read_csv(file_path)
    result_df.drop(result_df.columns[0], axis=1, inplace=True)
    return result_df

BLEU Score: Measures the similarity between the generated language response and the reference response based on n-gram overlap. Higher BLEU scores indicate greater similarity.

In [7]:
# @title
def compute_bleu(reference, candidate):
    reference = [[ref.split()] for ref in reference]
    candidate = [cand.split() for cand in candidate]
    smoothing_function = SmoothingFunction().method4
    bleu_scores = [corpus_bleu([ref], [cand], smoothing_function=smoothing_function) for ref, cand in zip(reference, candidate)]
    # Compute mean BLEU score and standard deviation
    mean_bleu_score = np.mean(bleu_scores)
    bleu_std_dev = np.std(bleu_scores)
    return round(mean_bleu_score, 3), round(bleu_std_dev, 3)


In [8]:
# @title
def compute_bleu_unigram(reference, candidate):
    reference = [[ref.split()] for ref in reference]
    candidate = [cand.split() for cand in candidate]
    smoothing_function = SmoothingFunction().method4
    weights = (1, 0, 0, 0)
    bleu_scores = [corpus_bleu([ref], [cand], smoothing_function=smoothing_function, weights = weights) for ref, cand in zip(reference, candidate)]
    # Compute mean BLEU score and standard deviation
    mean_bleu_score = round(np.mean(bleu_scores), 3)
    bleu_std_dev = round(np.std(bleu_scores), 3)
    return mean_bleu_score, bleu_std_dev

ROUGE Score (unigram and bigram): Calculates the overlap between the generated language response and the reference response at the unigram and bigram levels. Higher ROUGE scores indicate greater similarity.

In [9]:
# @title
def compute_rouge(reference, candidate):
    rouge = Rouge()

    # Compute ROUGE scores for all pairs of reference and candidate sentences
    
    rouge_scores = [rouge.get_scores(cand, ref, avg=True) for ref, cand in zip(reference, candidate)]
    

    # Extract individual ROUGE scores (f, p, r) for each pair
    rouge_1_f_scores = [score['rouge-1']['f'] for score in rouge_scores]
    rouge_2_f_scores = [score['rouge-2']['f'] for score in rouge_scores]
    rouge_l_f_scores = [score['rouge-l']['f'] for score in rouge_scores]

    # Compute mean and standard deviation of ROUGE F1 scores
    mean_rouge_1_f_score = np.mean(rouge_1_f_scores)
    mean_rouge_2_f_score = np.mean(rouge_2_f_scores)
    mean_rouge_l_f_score = np.mean(rouge_l_f_scores)

    rouge_1_f_std_dev = np.std(rouge_1_f_scores)
    rouge_2_f_std_dev = np.std(rouge_2_f_scores)
    rouge_l_f_std_dev = np.std(rouge_l_f_scores)

    return round(mean_rouge_1_f_score, 3), round(mean_rouge_2_f_score, 3), round(mean_rouge_l_f_score, 3), round(rouge_1_f_std_dev, 3), round(rouge_2_f_std_dev, 3), round(rouge_l_f_std_dev, 3)

BERTScore: Computes a similarity score for each token in the candidate sentence with each token in the reference sentence, based on contextual embeddings obtained from a pre-trained BERT model.

In [10]:
# @title
def compute_bert_score(reference, candidate):
    bert_p_scores, bert_r_scores, bert_f1_scores = bert_score.score(candidate, reference, lang="en", verbose=False)
    #print(bert_p_scores.mean().item(), bert_r_scores.mean().item())
    return round(bert_f1_scores.mean().item(), 3), round(bert_f1_scores.std().item(), 3)

METEOR Score: Evaluates the quality of the generated language response by considering both exact word matches and semantic similarity. Higher METEOR scores indicate better performance.

In [11]:
# @title
def compute_meteor_scores(reference, candidate):
  # Compute the METEOR scores for each candidate-reference pair
  # Tokenize each sentence
  tokenized_candidates = [word_tokenize(candidate.replace("<s>", "").replace("</s>", "").strip()) for candidate in candidate]
  tokenized_references = [word_tokenize(sentence) for sentence in reference]
  # Compute the METEOR scores for each candidate-reference pair
  meteor_scores = []
  for ref_sentence, candidate in zip(tokenized_references, tokenized_candidates):
      meteor_scores.append(single_meteor_score(ref_sentence, candidate))
  meteor_scores_mean = sum(meteor_scores) / len(meteor_scores)
  meteor_scores_std = statistics.stdev(meteor_scores)
  return round(meteor_scores_mean, 3), round(meteor_scores_std, 3)

In [12]:

import re

def clean_text(text):
    # Regular expression to match unwanted symbols and numbers
    regex = r"[^a-zA-Z0-9.,!?;:'\"()\[\]{}\-\s]"

    # Remove unwanted symbols and numbers
    cleaned_text = re.sub(regex, '', text)

    # Split text into lines
    lines = re.split(r'(?<=[.!?]) +', cleaned_text)

    # Remove duplicate lines while preserving order
    seen = set()
    unique_lines = []
    for line in lines:
        cleaned_line = re.sub(r'\s+', ' ', line).strip()  # Clean up extra spaces in each line
        if cleaned_line not in seen:
            seen.add(cleaned_line)
            unique_lines.append(cleaned_line)

    # Join lines back into a single string
    # Since these scores are for text similarity, restricting the generated caption to just 2 lines.
    unique_lines = unique_lines[:1]
    result = ' '.join(unique_lines)
    return result

def cleanup_pred_captions(predicted_captions):

    predicted_captions = predicted_captions.fillna('')
    clean_captions = []

    for caption in predicted_captions:
        # clean_caption = f"An image of {category}."
        clean_caption = f"No response."
        if caption.strip():
            clean_caption = clean_text(caption)
            if not clean_caption.strip():
              # clean_caption = f"An image of {category}."
              clean_caption = f"No response."
        clean_captions.append(clean_caption)

    return clean_captions

In [13]:
def run(csv_path):
    results = {}
    result_df = read_result_csv(csv_path)

    image_paths = result_df['Ground Truth Image']
    expected_captions = result_df['Expected Caption']
    predicted_captions = result_df['Generated Caption']
    expected_object_classes = result_df['Expected object']
    predicted_object_classes = result_df['Predicted object']

    predicted_captions = cleanup_pred_captions(predicted_captions)
    references = expected_captions.tolist()
    candidates = predicted_captions

    for i, cand in enumerate(candidates):
        if len(cand)<=1:
            candidates[i] = "No response"

    # BLEU Score

    mean_bleu_score, bleu_std_dev = compute_bleu(references, candidates)
    results["Mean BLEU Score"] =  mean_bleu_score
    results["SD BLEU Score"] =  bleu_std_dev

    mean_bleu_score, bleu_std_dev = compute_bleu_unigram(references, candidates)
    results["Mean BLEU Unigram Score"] =  mean_bleu_score
    results["SD BLEU Unigram Score"] =  bleu_std_dev

    # ROUGE Score

    mean_rouge_1_f_score, mean_rouge_2_f_score, mean_rouge_l_f_score, rouge_1_f_std_dev, rouge_2_f_std_dev, rouge_l_f_std_dev = compute_rouge(references, candidates)
    results["Mean ROUGE-1"] =  mean_rouge_1_f_score
    results["SD ROUGE-1"] = rouge_1_f_std_dev
    results["Mean ROUGE-2"] = mean_rouge_2_f_score
    results["SD ROUGE-2"] = rouge_2_f_std_dev
    results["Mean ROUGE-l"] = mean_rouge_l_f_score
    results["SD ROUGE-l"] =  rouge_l_f_std_dev

    # METEOR Score

    mean_meteor_score, meteor_std_dev = compute_meteor_scores(references, candidates)
    results["Mean Meteor Score"] =  mean_meteor_score
    results["SD Meteor Score"] = meteor_std_dev

    # BERT Score

    bert_score_mean, bert_score_std_dev = compute_bert_score(references, candidates)

    results["Mean BERTScore"] = round(bert_score_mean,3)
    results["SD BERTScore"] = round(bert_score_std_dev, 3)
    return results

In [14]:
from nltk.tokenize import sent_tokenize
import nltk

print(nltk.__version__)  # Check your NLTK version
nltk.download('punkt')

# Try a simple sentence split
print(sent_tokenize("This is a test. It should work correctly."))


3.8.1
['This is a test.', 'It should work correctly.']


[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# The csv_path below is the csv with generated text
# The below csv_file is for evaluation of text generated for Subject 1 EEG signals using Phi3 model.
# import os
# import tqdm
# results_dir = "../results"
# all_res = {}
# for file in tqdm.tqdm(os.listdir(results_dir)):
#     print (file)
#     fullpath = os.path.join(results_dir,file)
#     results = run(csv_path = fullpath)
#     all_res[file.replace("csv","")] = results

# results_df = pd.DataFrame(all_res).transpose()
# results_df.to_csv("all_results.csv") 
# The csv_path below is the csv with generated text
# The below csv_file is for evaluation of text generated for Subject 1 EEG signals using Phi3 model.
import os
import tqdm
results_dir = "results_experiment"
all_res = {}
for file in tqdm.tqdm(os.listdir(results_dir)):
    print (file)
    fullpath = os.path.join(results_dir,file)
    results = run(csv_path = fullpath)
    all_res[file.replace("csv","")] = results

results_df = pd.DataFrame(all_res).transpose()
results_df.to_csv("all_results.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

deepseek_eeg_model_7B_base_results.csv


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1/1 [00:54<00:00, 54.74s/it]
